In [1]:
%matplotlib inline
import pandas as pd

In [2]:
summary_df = pd.read_csv('all.summary.csv')

In [4]:
len(summary_df[summary_df['n_matches'] == 0])

2000

In [11]:
len(summary_df[summary_df['n_matches'] != 0])
nonempty_df = summary_df[summary_df['n_matches'] != 0]

In [8]:
len(summary_df[summary_df['p_weighted_covered'] > 90])

881

In [13]:
len(nonempty_df[nonempty_df['p_weighted_covered'] < 5])


1008

In [12]:
nonempty_df.describe()

,n_matches,p_weighted_covered
count,7988.000000,7988.000000
mean,273.413495,49.273748
std,455.414336,33.120600
min,1.000000,0.000000
25%,3.000000,16.100000
50%,64.000000,50.600000
75%,376.000000,81.600000
max,6215.000000,100.000000


In [7]:
summary_df.head()

,sample,n_matches,p_weighted_covered
0,DRR014176,373,88.1
1,DRR019507,97,4.3
2,DRR021939,1,68.0
3,DRR025071,1069,39.1
4,DRR033608,26,12.2
